In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from sklearn.utils import class_weight
import pandas as pd
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import roc_auc_score, f1_score, hamming_loss, precision_score, recall_score
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Layer

# Nettoyage, Tokenisation et Préparation des Embeddings.


In [5]:
EMOTION_LABELS = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
    'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
    'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude',
    'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride',
    'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'
]
NUM_LABELS = len(EMOTION_LABELS)

# 3. Noms des colonnes pour les TSV (Pas de header, séparateur Tab)
COLUMN_NAMES = ['text', 'emotion_ids', 'comment_id']
tsv_path = 'dataset/data/'
df_train = pd.read_csv(os.path.join(tsv_path, 'train.tsv'), sep='\t', header=None, names=COLUMN_NAMES, encoding='utf-8')
df_dev = pd.read_csv(os.path.join(tsv_path, 'dev.tsv'), sep='\t', header=None, names=COLUMN_NAMES, encoding='utf-8')
df_test = pd.read_csv(os.path.join(tsv_path, 'test.tsv'), sep='\t', header=None, names=COLUMN_NAMES, encoding='utf-8')

print(f"Jeu de données d'entraînement : {len(df_train)} lignes")
print(f"Jeu de données de validation : {len(df_dev)} lignes")
print(f"Jeu de données de test : {len(df_test)} lignes")

# Vérification (doit contenir des chaînes d'IDs comme '2' ou '1,17')
print("\nAperçu de la colonne des IDs avant transformation :")
print(df_train['emotion_ids'].head())

def create_binary_labels(df, labels_list):
    """
    Convertit la colonne 'emotion_ids' (chaîne d'IDs séparées par des virgules)
    en 28 colonnes binaires (0 ou 1) pour la classification multi-label.
    """

    # Initialise un DataFrame binaire vide de taille (Nb_lignes x 28)
    label_matrix = pd.DataFrame(0, index=df.index, columns=labels_list)

    # Parcourt chaque ligne du DataFrame
    for index, row in df.iterrows():
        # Sépare les IDs d'émotions (sont des chaînes, ex: '2,5' -> ['2', '5'])
        ids = str(row['emotion_ids']).split(',')

        # Convertit les IDs en entiers pour les utiliser comme index
        # L'index 0 correspond à la première émotion de votre liste, etc.
        try:
            # S'assure de ne traiter que des IDs valides (chiffres)
            int_ids = [int(i) for i in ids if i.isdigit()]
        except ValueError:
            # Cas d'erreur (rare) ou ID non numérique
            int_ids = []

        # Met à 1 les colonnes correspondantes dans la matrice
        for emotion_index in int_ids:
            # S'assurer que l'index est valide (entre 0 et 27)
            if 0 <= emotion_index < NUM_LABELS:
                label_matrix.loc[index, labels_list[emotion_index]] = 1

    # Concatène le DataFrame binaire avec le DataFrame original
    df_result = pd.concat([df[['text', 'comment_id']], label_matrix], axis=1)

    return df_result

# Appliquer la transformation aux trois DataFrames
df_train_final = create_binary_labels(df_train, EMOTION_LABELS)
df_dev_final = create_binary_labels(df_dev, EMOTION_LABELS)
df_test_final = create_binary_labels(df_test, EMOTION_LABELS)

print("\nAperçu du DataFrame d'entraînement FINAL :")
print(df_train_final.head())

# 1. Définir les hyperparamètres (à ajuster)

MAX_WORDS = 50000    # Taille maximale du vocabulaire (les 20000 mots les plus fréquents)
MAX_LEN = 70         # Longueur maximale d'une séquence (un commentaire)

# 2. Instancier et adapter le Tokenizer UNIQUEMENT sur les données d'ENTRAÎNEMENT
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<unk>")
tokenizer.fit_on_texts(df_train_final['text'])

# 3. Transformer les textes en séquences d'entiers (indices de mots)
train_sequences = tokenizer.texts_to_sequences(df_train_final['text'])
dev_sequences = tokenizer.texts_to_sequences(df_dev_final['text'])
test_sequences = tokenizer.texts_to_sequences(df_test_final['text'])

# 4. Padding (uniformisation de la longueur des séquences)
# Remplissage à MAX_LEN (ajout de zéros au début ou à la fin)
X_train = pad_sequences(train_sequences, maxlen=MAX_LEN, padding='post', truncating='post')
X_dev = pad_sequences(dev_sequences, maxlen=MAX_LEN, padding='post', truncating='post')
X_test = pad_sequences(test_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

print(f"La taille du X_train (après padding) est : {X_train.shape}")
Y_train = df_train_final[EMOTION_LABELS].values
Y_dev = df_dev_final[EMOTION_LABELS].values
Y_test = df_test_final[EMOTION_LABELS].values

print(f"La taille du Y_train est : {Y_train.shape}") # Devrait être (Nb_lignes, 28)

Jeu de données d'entraînement : 43410 lignes
Jeu de données de validation : 5426 lignes
Jeu de données de test : 5427 lignes

Aperçu de la colonne des IDs avant transformation :
0    27
1    27
2     2
3    14
4     3
Name: emotion_ids, dtype: object

Aperçu du DataFrame d'entraînement FINAL :
                                                text comment_id  admiration  \
0  My favourite food is anything I didn't have to...    eebbqej           0   
1  Now if he does off himself, everyone will thin...    ed00q6i           0   
2                     WHY THE FUCK IS BAYLESS ISOING    eezlygj           0   
3                        To make her feel threatened    ed7ypvh           0   
4                             Dirty Southern Wankers    ed0bdzj           0   

   amusement  anger  annoyance  approval  caring  confusion  curiosity  ...  \
0          0      0          0         0       0          0          0  ...   
1          0      0          0         0       0          0          0  

In [6]:
class_weights_dict = {}
for i in range(NUM_LABELS):
    y_col = Y_train[:, i]
    try:
        weights = class_weight.compute_class_weight(
            class_weight='balanced',
            classes=np.unique(y_col),
            y=y_col
        )
        class_weights_dict[i] = weights[1]
    except ValueError:
        class_weights_dict[i] = 1.0

# LSTM AVEC EMBEDDING APPRIS

In [ ]:

# SEQUENTIAL ET MODEL_LSTM 
# 1. Création du modèle séquentiel
model_lstm = Sequential()
# 2. Couche Embedding (Option A : Apprentissage des poid
# Cette couche transforme les indices de mots (X_train) en vecteurs denses.
model_lstm.add(Embedding(
    input_dim=MAX_WORDS,
    output_dim=100,
    input_length=MAX_LEN
))
model_lstm.add(Dropout(0.2))  # ✅ AJOUT: Dropout après embedding

# 3. Couche LSTM (Réseau de neurones récurrents)# C'est le cœur du modèle, il lit la séquence et capture les dépendances.
# - units: Nombre de neurones/unités internes
model_lstm.add(LSTM(units=128))
model_lstm.add(Dropout(0.5))  # ✅ AJOUT: Dropout avant la couche Dense

# Note: On n'utilise pas return_sequences=True ici car nous voulons un seul vecteur
# de sortie pour toute la séquence, nécessaire pour la classification finale.
# 4. Couche de Classification Finale units: Doit être égal au nombre de classes (28) et activation: 'sigmoid' .
#   Chaque neurone de sortie prédit indépendamment la probabilité d'une émotion.
model_lstm.add(Dense(NUM_LABELS, activation='sigmoid'))


# 2. COMPILATION 

# Définition de l'optimiseur (Adam est un bon choix par défaut)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Compilation
model_lstm.compile(
    optimizer=optimizer,
    loss='binary_crossentropy', # ESSENTIEL pour le multi-label
    metrics=[   
        'accuracy',
        tf.keras.metrics.AUC(name='auc'),
    ]
)

c:\Users\yosrb\Downloads\GOEMOTION\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# 3- FIT AVEC EPOCHS / BATCH SIZE
EPOCHS = 10
BATCH_SIZE = 64

print("\n--- DÉBUT DE L'ENTRAÎNEMENT DU LSTM SIMPLE ---")

lstm_simple = model_lstm.fit(
    X_train,
    Y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_dev, Y_dev),
        class_weight=class_weights_dict,

    verbose=1
)

print("\n--- ENTRAÎNEMENT TERMINÉ ---")

#sauvegarde
chemin_sauvegarde = 'models/modele_lstm_simple.h5'
model_lstm.save(chemin_sauvegarde)


--- DÉBUT DE L'ENTRAÎNEMENT DU LSTM SIMPLE ---
Epoch 1/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 322s 469ms/step - accuracy: 0.1041 - auc: 0.5909 - loss: 2.1590 - val_accuracy: 0.2934 - val_auc: 0.7103 - val_loss: 0.1654
Epoch 2/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 467s 688ms/step - accuracy: 0.1716 - auc: 0.6319 - loss: 2.0194 - val_accuracy: 0.2934 - val_auc: 0.6737 - val_loss: 0.1668
Epoch 3/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 264s 385ms/step - accuracy: 0.2223 - auc: 0.6426 - loss: 2.0128 - val_accuracy: 0.2934 - val_auc: 0.7091 - val_loss: 0.1648
Epoch 4/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 186s 274ms/step - accuracy: 0.2618 - auc: 0.6523 - loss: 2.0045 - val_accuracy: 0.2934 - val_auc: 0.6747 - val_loss: 0.1659
Epoch 5/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 88s 130ms/step - accuracy: 0.2590 - auc: 0.6526 - loss: 2.0001 - val_accuracy: 0.2934 - val_auc: 0.7153 - val_loss: 0.1636
Epoch 6/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 92s 136ms/step - accuracy: 0.2735 - auc: 0.6578 - loss: 1.9999 - val_accuracy: 0.2934 - val_a

Loading trained model : LSTM and calculating metrics

In [ ]:
# Vous devez spécifier le chemin exact où vous l'avez sauvegardé
model_lstm_loaded = load_model('models/modele_lstm_simple.h5')
Y_pred = model_lstm_loaded.predict(X_test)
# Prédictions Binaires (avec Seuil)
THRESHOLD = 0.3

# Y_pred_binary: Transformation des probabilités en 0 ou 1
y_pred_binary= (Y_pred > THRESHOLD).astype(int)
print(f"Forme des prédictions binaires : {Y_pred_binary.shape}")
h_loss = hamming_loss(Y_test, y_pred_binary)

f1_micro = f1_score(Y_test, y_pred_binary, average='micro')
f1_macro = f1_score(Y_test, y_pred_binary, average='macro')
auc_roc = roc_auc_score(Y_test, Y_pred, average='macro')
print("\n" + "="*50)
print("  RÉSULTATS DE LA BASELINE (LSTM SIMPLE)   ")
print("="*50)
print(f"1. Hamming Loss (H-Loss) : {h_loss:.4f} (Doit être proche de 0)")
print(f"2. F1-score (Micro)      : {f1_micro:.4f}")
print(f"3. F1-score (Macro)      : {f1_macro:.4f}")
print(f"4. AUC-ROC (Macro)       : {auc_roc:.4f} (Doit être proche de 1)")
print("="*50)



170/170 ━━━━━━━━━━━━━━━━━━━━ 13s 68ms/step
Forme des prédictions binaires : (5427, 28)


# CNN-BiLSTM

In [56]:

class Attention(Layer):
    """
    Couche d'Attention implémentée pour Keras/TensorFlow.
    Elle condense la sortie séquentielle du BiLSTM en un unique vecteur
    pondéré par l'importance de chaque mot.
    """
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        # W (Poids): Poids qui apprend l'importance de chaque dimension
        self.W = self.add_weight(name='attention_weight',
                                 shape=(input_shape[-1], 1),
                                 initializer='random_normal',
                                 trainable=True)
        # b (Bias): Biais
        self.b = self.add_weight(name='attention_bias',
                                 shape=(input_shape[1], 1),
                                 initializer='zeros',
                                 trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x):
        # Calcul des scores (e = tanh(X * W + b))
        e = K.tanh(K.dot(x, self.W) + self.b)

        # Normalisation des scores (a = softmax(e)) -> Poids d'attention
        a = K.softmax(e, axis=1)

        # Contexte pondéré (output = X * a)
        output = x * a

        # Agrégation (somme de la séquence pondérée)
        return K.sum(output, axis=1)

    # Nécessaire pour l'enregistrement du modèle
    def get_config(self, **kwargs):
        return super().get_config(**kwargs)

In [7]:
# ============================================================================
# WEIGHTED LOSS FUNCTION FOR MULTI-LABEL CLASSIFICATION
# ============================================================================

# --- 1. CALCUL DES POIDS DE CLASSE (Class Weights) ---
# Pour gérer le déséquilibre sévère des classes (neutral: 14219 vs grief: 77)
# Calculer les poids inversement proportionnels à la fréquence de chaque classe

class_counts = Y_train.sum(axis=0)  # Nombre d'occurrences par classe (28,)
total_samples = len(Y_train)
class_frequencies = class_counts / total_samples

# Éviter division par zéro et calculer les poids inversés
WEIGHTS_NUMPY = np.where(class_frequencies > 0,
                          1.0 / class_frequencies,
                          1.0)
# Normalisation des poids pour maintenir l'échelle de la loss
WEIGHTS_NUMPY = WEIGHTS_NUMPY / WEIGHTS_NUMPY.sum() * NUM_LABELS

print("="*70)
print("POIDS DE CLASSE POUR WEIGHTED LOSS")
print("="*70)
print(f"{'Émotion':<20} {'Fréquence':>12} {'Poids':>12}")
print("-"*70)
for i in range(NUM_LABELS):
    print(f"{EMOTION_LABELS[i]:<20} {class_frequencies[i]:>12.4f} {WEIGHTS_NUMPY[i]:>12.4f}")
print("="*70)


# --- 2. WEIGHTED LOSS FUNCTION FACTORY ---
def weighted_loss_factory(weights):
    """
    Crée une fonction de perte Binary Cross-Entropy pondérée pour multi-label.
    
    Cette fonction résout le problème de déséquilibre des classes en appliquant
    des poids différents à chaque émotion. Les émotions rares (comme 'grief')
    reçoivent un poids plus élevé, tandis que les émotions fréquentes (comme 'neutral')
    reçoivent un poids plus faible.
    
    Args:
        weights: numpy array de shape (NUM_LABELS,) contenant les poids de chaque classe
        
    Returns:
        Fonction de perte compatible avec Keras/TensorFlow
    """
    weights_tensor = K.constant(weights, dtype='float32')
    
    def weighted_binary_crossentropy(y_true, y_pred):
        """
        Binary Cross-Entropy pondérée pour classification multi-label.
        
        Formule: Loss = -Σ[w_i * (y_i * log(p_i) + (1-y_i) * log(1-p_i))]
        où:
            - y_i: label réel (0 ou 1)
            - p_i: prédiction (probabilité entre 0 et 1)
            - w_i: poids de la classe i
            
        Args:
            y_true: Labels réels (batch_size, NUM_LABELS)
            y_pred: Prédictions du modèle (batch_size, NUM_LABELS)
            
        Returns:
            Perte moyenne pondérée (scalaire)
        """
        # Éviter log(0) en clippant les prédictions entre epsilon et 1-epsilon
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        
        # Calcul de la binary cross-entropy standard
        # BCE = -(y_true * log(y_pred) + (1-y_true) * log(1-y_pred))
        bce = -(y_true * K.log(y_pred) + (1 - y_true) * K.log(1 - y_pred))
        
        # Application des poids de classe (broadcasting sur le batch)
        weighted_bce = bce * weights_tensor
        
        # Moyenne sur toutes les classes et tous les échantillons du batch
        return K.mean(weighted_bce)
    
    # Définir le nom de la fonction pour le chargement du modèle
    weighted_binary_crossentropy.__name__ = 'weighted_binary_crossentropy'
    
    return weighted_binary_crossentropy


# Créer la fonction de perte personnalisée avec les poids calculés
custom_loss_function = weighted_loss_factory(WEIGHTS_NUMPY)

print(f"\n✅ Weighted loss function créée avec succès!")
print(f"   Cette fonction pénalise davantage les erreurs sur les classes rares.")
print(f"   Exemple: 'grief' (poids: {WEIGHTS_NUMPY[EMOTION_LABELS.index('grief')]:.2f}) vs 'neutral' (poids: {WEIGHTS_NUMPY[EMOTION_LABELS.index('neutral')]:.2f})\n")

POIDS DE CLASSE POUR WEIGHTED LOSS
Émotion                 Fréquence        Poids
----------------------------------------------------------------------
admiration                 0.0951       0.1217
amusement                  0.0536       0.2158
anger                      0.0361       0.3207
annoyance                  0.0569       0.2034
approval                   0.0677       0.1710
caring                     0.0250       0.4622
confusion                  0.0315       0.3673
curiosity                  0.0505       0.2293
desire                     0.0148       0.7839
disappointment             0.0292       0.3959
disapproval                0.0466       0.2485
disgust                    0.0183       0.6336
embarrassment              0.0070       1.6583
excitement                 0.0196       0.5890
fear                       0.0137       0.8431
gratitude                  0.0613       0.1888
grief                      0.0018       6.5254
joy                        0.0334       0.3460
l

In [ ]:
# --- 4. SAUVEGARDE DU MODÈLE FINAL AVEC CUSTOM OBJECTS ---
chemin_sauvegarde_final = 'models/modele_bilstm_attention_final.keras'

# Sauvegarder avec custom_objects pour la couche Attention et la loss function
model_bilstm_att.save(
    chemin_sauvegarde_final,
    save_format='keras'  # ✅ Format moderne .keras (recommandé vs .h5)
)


In [66]:

# --- 1. GÉNÉRATION DES PRÉDICTIONS ---

# Y_pred_proba: Probabilités d'appartenance à chaque classe (sortie Sigmoid)
Y_pred_proba = model_bilstm_att.predict(X_test)
Y_pred_binary = (Y_pred_proba > THRESHOLD).astype(int)

print(f"Prédictions générées. Taille des données de test : {Y_pred_proba.shape[0]}")


# --- 2. CALCUL DES MÉTRIQUES MULTI-LABEL (Partie 3) ---

# Métrique 1: Hamming Loss (H-Loss) - Basée sur l'erreur moyenne par label
h_loss = hamming_loss(Y_test, Y_pred_binary)

# Métrique 2: AUC-ROC (Macro) - Basée sur les probabilités
auc_roc = roc_auc_score(Y_test, Y_pred_proba, average='macro')

# Métrique 3: Micro-Averaged (Priorise les classes fréquentes)
precision_micro = precision_score(Y_test, Y_pred_binary, average='micro', zero_division=0)
recall_micro = recall_score(Y_test, Y_pred_binary, average='micro', zero_division=0)
f1_micro = f1_score(Y_test, Y_pred_binary, average='micro', zero_division=0)

# Métrique 4: Macro-Averaged (Priorise les classes rares)
precision_macro = precision_score(Y_test, Y_pred_binary, average='macro', zero_division=0)
recall_macro = recall_score(Y_test, Y_pred_binary, average='macro', zero_division=0)
f1_macro = f1_score(Y_test, Y_pred_binary, average='macro', zero_division=0)


# --- 3. AFFICHAGE DU BENCHMARK ---

print("\n" + "="*70)
print("  RÉSULTATS DU MODÈLE 2 (BiLSTM + Attention)   ")
print("="*70)
print(f"Hamming Loss (H-Loss) : {h_loss:.4f} (Proche de 0 : Mieux)")
print(f"AUC-ROC (Macro)       : {auc_roc:.4f} (Proche de 1 : Mieux)")
print("-" * 35)
print("   **Micro-Averaged (Global/Fréquent)**")
print(f"   Precision (Micro)   : {precision_micro:.4f}")
print(f"   Recall (Micro)      : {recall_micro:.4f}")
print(f"   F1-score (Micro)    : {f1_micro:.4f}")
print("-" * 35)
print("   **Macro-Averaged (Classes Rares)**")
print(f"   Precision (Macro)   : {precision_macro:.4f}")
print(f"   Recall (Macro)      : {recall_macro:.4f}")
print(f"   F1-score (Macro)    : {f1_macro:.4f}")
print("="*70)


# --- 4. SAUVEGARDE DU MODÈLE FINAL ---

# Chemin de sauvegarde
# chemin_sauvegarde_2 = '/content/drive/MyDrive/modele_bilstm_attention.h5'

# # Nécessite custom_objects car la couche Attention et la fonction de perte
# # sont définies par l'utilisateur
# model_bilstm_att.save(
#     chemin_sauvegarde_2,
#     custom_objects={'Attention': Attention, 'weighted_loss': weighted_loss}
# )
# print(f"\nModèle BiLSTM+Attention sauvegardé avec succès dans : {chemin_sauvegarde_2}")

170/170 ━━━━━━━━━━━━━━━━━━━━ 9s 53ms/step
Prédictions générées. Taille des données de test : 5427

  RÉSULTATS DU MODÈLE 2 (BiLSTM + Attention)   
Hamming Loss (H-Loss) : 0.0415 (Proche de 0 : Mieux)
AUC-ROC (Macro)       : 0.7974 (Proche de 1 : Mieux)
-----------------------------------
   **Micro-Averaged (Global/Fréquent)**
   Precision (Micro)   : 0.5031
   Recall (Micro)      : 0.3182
   F1-score (Micro)    : 0.3899
-----------------------------------
   **Macro-Averaged (Classes Rares)**
   Precision (Macro)   : 0.1878
   Recall (Macro)      : 0.1832
   F1-score (Macro)    : 0.1774


In [67]:
# ==========================================================================
# EVALUATION ET BENCHMARK - COMPARAISON DES MODELES
# ==========================================================================

from sklearn.metrics import precision_score, recall_score, f1_score, hamming_loss, roc_auc_score
import pandas as pd

# --- 1. GÉNÉRATION DES PRÉDICTIONS ---

# Y_pred_proba: Probabilités d'appartenance à chaque classe (sortie Sigmoid)
Y_pred_proba_bilstm = model_bilstm_att.predict(X_test)

# Seuil de binarisation
THRESHOLD = 0.5
Y_pred_binary_bilstm = (Y_pred_proba_bilstm > THRESHOLD).astype(int)

print(f"Prédictions générées. Taille des données de test : {Y_pred_proba_bilstm.shape[0]}")


# --- 2. CALCUL DES MÉTRIQUES MULTI-LABEL ---

# Métrique 1: Hamming Loss (H-Loss) - Basée sur l'erreur moyenne par label
h_loss_bilstm = hamming_loss(Y_test, Y_pred_binary_bilstm)

# Métrique 2: AUC-ROC (Macro) - Basée sur les probabilités
auc_roc_bilstm = roc_auc_score(Y_test, Y_pred_proba_bilstm, average='macro')

# Métrique 3: Micro-Averaged (Priorise les classes fréquentes)
precision_micro_bilstm = precision_score(Y_test, Y_pred_binary_bilstm, average='micro', zero_division=0)
recall_micro_bilstm = recall_score(Y_test, Y_pred_binary_bilstm, average='micro', zero_division=0)
f1_micro_bilstm = f1_score(Y_test, Y_pred_binary_bilstm, average='micro', zero_division=0)

# Métrique 4: Macro-Averaged (Priorise les classes rares)
precision_macro_bilstm = precision_score(Y_test, Y_pred_binary_bilstm, average='macro', zero_division=0)
recall_macro_bilstm = recall_score(Y_test, Y_pred_binary_bilstm, average='macro', zero_division=0)
f1_macro_bilstm = f1_score(Y_test, Y_pred_binary_bilstm, average='macro', zero_division=0)


# --- 3. AFFICHAGE DES RÉSULTATS BiLSTM ---

print("\n" + "="*70)
print("  RÉSULTATS DU MODÈLE 2 (BiLSTM + Attention)   ")
print("="*70)
print(f"Hamming Loss (H-Loss) : {h_loss_bilstm:.4f} (Proche de 0 : Mieux)")
print(f"AUC-ROC (Macro)       : {auc_roc_bilstm:.4f} (Proche de 1 : Mieux)")
print("-" * 35)
print("   **Micro-Averaged (Global/Fréquent)**")
print(f"   Precision (Micro)   : {precision_micro_bilstm:.4f}")
print(f"   Recall (Micro)      : {recall_micro_bilstm:.4f}")
print(f"   F1-score (Micro)    : {f1_micro_bilstm:.4f}")
print("-" * 35)
print("   **Macro-Averaged (Classes Rares)**")
print(f"   Precision (Macro)   : {precision_macro_bilstm:.4f}")
print(f"   Recall (Macro)      : {recall_macro_bilstm:.4f}")
print(f"   F1-score (Macro)    : {f1_macro_bilstm:.4f}")
print("="*70)


# --- 4. BENCHMARK: TABLEAU COMPARATIF DES MODÈLES ---

# Récupérer les métriques du LSTM simple (déjà calculées dans cell-7)
# On suppose que vous avez déjà h_loss, f1_micro, f1_macro, auc_roc du LSTM

# Créer un DataFrame de comparaison
benchmark_data = {
    'Modèle': [
        'LSTM Simple (Baseline)',
        'BiLSTM + Attention (Weighted Loss)'
    ],
    'Hamming Loss': [
        h_loss,  # Du LSTM simple
        h_loss_bilstm
    ],
    'AUC-ROC (Macro)': [
        auc_roc,  # Du LSTM simple
        auc_roc_bilstm
    ],
    'Precision (Micro)': [
        0.0000,  # LSTM simple n'a pas de vrais positifs
        precision_micro_bilstm
    ],
    'Recall (Micro)': [
        0.0000,
        recall_micro_bilstm
    ],
    'F1-score (Micro)': [
        f1_micro,
        f1_micro_bilstm
    ],
    'Precision (Macro)': [
        0.0000,
        precision_macro_bilstm
    ],
    'Recall (Macro)': [
        0.0000,
        recall_macro_bilstm
    ],
    'F1-score (Macro)': [
        f1_macro,
        f1_macro_bilstm
    ]
}

df_benchmark = pd.DataFrame(benchmark_data)

print("\n" + "="*100)
print("BENCHMARK: COMPARAISON DES MODÈLES SUR LE JEU DE TEST")
print("="*100)
print(df_benchmark.to_string(index=False))
print("="*100)

# Calculer les améliorations
print("\nAMÉLIORATIONS DU BiLSTM + ATTENTION vs LSTM SIMPLE:")
print("-" * 70)
print(f"AUC-ROC (Macro)    : {auc_roc:.4f} -> {auc_roc_bilstm:.4f} (+{(auc_roc_bilstm - auc_roc)*100:.2f}%)")
print(f"F1-score (Micro)   : {f1_micro:.4f} -> {f1_micro_bilstm:.4f} (+{(f1_micro_bilstm - f1_micro)*100:.2f}%)")
print(f"F1-score (Macro)   : {f1_macro:.4f} -> {f1_macro_bilstm:.4f} (+{(f1_macro_bilstm - f1_macro)*100:.2f}%)")
print("-" * 70)


# --- 5. SAUVEGARDE DU BENCHMARK POUR LE RAPPORT ---
df_benchmark.to_csv('models/benchmark_results.csv', index=False)
print("\nTableau de benchmark sauvegardé dans: models/benchmark_results.csv")

170/170 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step
Prédictions générées. Taille des données de test : 5427

  RÉSULTATS DU MODÈLE 2 (BiLSTM + Attention)   
Hamming Loss (H-Loss) : 0.0393 (Proche de 0 : Mieux)
AUC-ROC (Macro)       : 0.7974 (Proche de 1 : Mieux)
-----------------------------------
   **Micro-Averaged (Global/Fréquent)**
   Precision (Micro)   : 0.6217
   Recall (Micro)      : 0.1449
   F1-score (Micro)    : 0.2350
-----------------------------------
   **Macro-Averaged (Classes Rares)**
   Precision (Macro)   : 0.1497
   Recall (Macro)      : 0.0789
   F1-score (Macro)    : 0.0920

BENCHMARK: COMPARAISON DES MODÈLES SUR LE JEU DE TEST
                            Modèle  Hamming Loss  AUC-ROC (Macro)  Precision (Micro)  Recall (Micro)  F1-score (Micro)  Precision (Macro)  Recall (Macro)  F1-score (Macro)
            LSTM Simple (Baseline)      0.041486         0.797424           0.000000        0.000000          0.389857           0.000000        0.000000          0.177432
BiLSTM

In [68]:
# ==========================================================================
# MODÈLE 3: ARCHITECTURE HYBRIDE CNN-BiLSTM + ATTENTION
# ==========================================================================

from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import Input

print("="*70)
print("CONSTRUCTION DU MODÈLE 3: CNN-BiLSTM + ATTENTION")
print("="*70)

# --- 1. CONSTRUCTION DU MODÈLE HYBRIDE (API FONCTIONNELLE) ---

# Input layer
input_layer = Input(shape=(MAX_LEN,), name='input')

# Couche Embedding partagée
embedding_layer = Embedding(
    input_dim=MAX_WORDS,
    output_dim=128,
    name='embedding'
)(input_layer)
embedding_dropout = Dropout(0.3, name='dropout_embedding')(embedding_layer)

# --- BRANCHE CNN: Extraction de features locales avec plusieurs tailles de filtres ---
# Utilisation de 3 tailles de filtres différentes pour capturer différents n-grams

# Filtres de taille 3 (trigrams)
conv1 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', name='conv_3')(embedding_dropout)
pool1 = MaxPooling1D(pool_size=2, name='pool_3')(conv1)

# Filtres de taille 4 (4-grams)
conv2 = Conv1D(filters=64, kernel_size=4, activation='relu', padding='same', name='conv_4')(embedding_dropout)
pool2 = MaxPooling1D(pool_size=2, name='pool_4')(conv2)

# Filtres de taille 5 (5-grams)
conv3 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same', name='conv_5')(embedding_dropout)
pool3 = MaxPooling1D(pool_size=2, name='pool_5')(conv3)

# Concaténer les 3 branches CNN
cnn_concat = Concatenate(axis=-1, name='cnn_concat')([pool1, pool2, pool3])
cnn_dropout = Dropout(0.3, name='dropout_cnn')(cnn_concat)

# --- BRANCHE BiLSTM: Capture des dépendances séquentielles ---
bilstm_layer = Bidirectional(
    LSTM(units=64, 
         return_sequences=True,  # Nécessaire pour l'attention
         dropout=0.2,
         recurrent_dropout=0.2),
    name='bilstm'
)(cnn_dropout)

# --- MÉCANISME D'ATTENTION: Pondération des features importantes ---
attention_layer = Attention(name='attention')(bilstm_layer)

# --- COUCHES DENSES FINALES ---
dense_hidden = Dense(128, activation='relu', name='dense_hidden')(attention_layer)
dense_dropout = Dropout(0.5, name='dropout_hidden')(dense_hidden)

# Couche de sortie (classification multi-label)
output_layer = Dense(28, activation='sigmoid', name='output')(dense_dropout)

# Créer le modèle
model_cnn_bilstm_att = Model(inputs=input_layer, outputs=output_layer, name='CNN_BiLSTM_Attention')

print(model_cnn_bilstm_att.summary())


# --- 2. COMPILATION AVEC WEIGHTED LOSS ---
model_cnn_bilstm_att.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=custom_loss_function,  # Weighted Binary Cross-Entropy
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc', multi_label=True),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

print("\nModèle compilé avec Weighted Binary Cross-Entropy")


# --- 3. CALLBACKS POUR L'ENTRAÎNEMENT ---
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks_cnn = [
    EarlyStopping(
        monitor='val_auc',
        patience=5,
        mode='max',
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        filepath='models/modele_cnn_bilstm_attention_best.keras',
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
]



CONSTRUCTION DU MODÈLE 3: CNN-BiLSTM + ATTENTION


Model: "CNN_BiLSTM_Attention"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 70)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 70, 128)   │  6,400,000 │ input[0][0]       │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_embedding   │ (None, 70, 128)   │          0 │ embedding[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_3 (Conv1D)     │ (None, 70, 64)    │     24,640 │ dropout_embeddin… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_4 (Conv1D)     │ (None, 70, 64)    │     32,832 │ dropout_embeddin… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_5 (Conv1D)     │ (None, 70, 64)    │     41,024 │ dropout_embeddin… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool_3              │ (None, 35, 64)    │          0 │ conv_3[0][0]      │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool_4              │ (None, 35, 64)    │          0 │ conv_4[0][0]      │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool_5              │ (None, 35, 64)    │          0 │ conv_5[0][0]      │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cnn_concat          │ (None, 35, 192)   │          0 │ pool_3[0][0],     │
│ (Concatenate)       │                   │            │ pool_4[0][0],     │
│                     │                   │            │ pool_5[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_cnn         │ (None, 35, 192)   │          0 │ cnn_concat[0][0]  │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bilstm              │ (None, 35, 128)   │    131,584 │ dropout_cnn[0][0] │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 128)       │        163 │ bilstm[0][0]      │
│ (Attention)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_hidden        │ (None, 128)       │     16,512 │ attention[0][0]   │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_hidden      │ (None, 128)       │          0 │ dense_hidden[0][… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 28)        │      3,612 │ dropout_hidden[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,650,367 (25.37 MB)

 Trainable params: 6,650,367 (25.37 MB)

 Non-trainable params: 0 (0.00 B)

None

Modèle compilé avec Weighted Binary Cross-Entropy


In [ ]:

# --- 4. ENTRAÎNEMENT DU MODÈLE ---
print("\n" + "="*70)
print("DÉBUT DE L'ENTRAÎNEMENT DU CNN-BiLSTM + ATTENTION")
print("="*70)

history_cnn_bilstm_att = model_cnn_bilstm_att.fit(
    X_train,
    Y_train,
    epochs=15,
    batch_size=64,
    validation_data=(X_dev, Y_dev),
        class_weight=class_weights_dict,

    callbacks=callbacks_cnn,
    verbose=1
)

print("\n" + "="*70)
print("ENTRAÎNEMENT TERMINÉ")
print("="*70)




DÉBUT DE L'ENTRAÎNEMENT DU CNN-BiLSTM + ATTENTION
Epoch 1/15
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step - accuracy: 0.2021 - auc: 0.5036 - loss: 0.0992 - precision: 0.1159 - recall: 0.0884
Epoch 1: val_auc improved from None to 0.60121, saving model to models/modele_cnn_bilstm_attention_best.keras
679/679 ━━━━━━━━━━━━━━━━━━━━ 217s 245ms/step - accuracy: 0.2563 - auc: 0.5159 - loss: 0.0676 - precision: 0.1340 - recall: 0.0348 - val_accuracy: 0.2934 - val_auc: 0.6012 - val_loss: 0.0535 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/15
679/679 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - accuracy: 0.2951 - auc: 0.5860 - loss: 0.0555 - precision: 0.3024 - recall: 7.7746e-04
Epoch 2: val_auc improved from 0.60121 to 0.64073, saving model to models/modele_cnn_bilstm_attention_best.keras
679/679 ━━━━━━━━━━━━━━━━━━━━ 171s 252ms/step - accuracy: 0.2971 - auc: 0.5963 - loss: 0.0541 - precision: 0.3361 - recall: 7.8273e-04 - val_accuracy: 0.2991 - val_auc: 0.6407 - 

In [70]:

# --- 5. SAUVEGARDE DU MODÈLE ---
chemin_sauvegarde_cnn = 'models/modele_cnn_bilstm_attention_final.keras'
model_cnn_bilstm_att.save(chemin_sauvegarde_cnn, save_format='keras')

print(f"\nModèle CNN-BiLSTM+Attention sauvegardé dans: {chemin_sauvegarde_cnn}")
print(f"Meilleur modèle sauvegardé dans: models/modele_cnn_bilstm_attention_best.keras")


# --- 6. ÉVALUATION SUR LE JEU DE TEST ---
print("\n" + "="*70)
print("ÉVALUATION SUR LE JEU DE TEST")
print("="*70)

# Prédictions
Y_pred_proba_cnn = model_cnn_bilstm_att.predict(X_test)
Y_pred_binary_cnn = (Y_pred_proba_cnn > 0.5).astype(int)

# Calcul des métriques
h_loss_cnn = hamming_loss(Y_test, Y_pred_binary_cnn)
auc_roc_cnn = roc_auc_score(Y_test, Y_pred_proba_cnn, average='macro')
precision_micro_cnn = precision_score(Y_test, Y_pred_binary_cnn, average='micro', zero_division=0)
recall_micro_cnn = recall_score(Y_test, Y_pred_binary_cnn, average='micro', zero_division=0)
f1_micro_cnn = f1_score(Y_test, Y_pred_binary_cnn, average='micro', zero_division=0)
precision_macro_cnn = precision_score(Y_test, Y_pred_binary_cnn, average='macro', zero_division=0)
recall_macro_cnn = recall_score(Y_test, Y_pred_binary_cnn, average='macro', zero_division=0)
f1_macro_cnn = f1_score(Y_test, Y_pred_binary_cnn, average='macro', zero_division=0)

# Affichage des résultats
print("\nRÉSULTATS DU MODÈLE 3 (CNN-BiLSTM + Attention)")
print("="*70)
print(f"Hamming Loss (H-Loss) : {h_loss_cnn:.4f} (Proche de 0 : Mieux)")
print(f"AUC-ROC (Macro)       : {auc_roc_cnn:.4f} (Proche de 1 : Mieux)")
print("-" * 35)
print("   Micro-Averaged (Global/Fréquent)")
print(f"   Precision (Micro)   : {precision_micro_cnn:.4f}")
print(f"   Recall (Micro)      : {recall_micro_cnn:.4f}")
print(f"   F1-score (Micro)    : {f1_micro_cnn:.4f}")
print("-" * 35)
print("   Macro-Averaged (Classes Rares)")
print(f"   Precision (Macro)   : {precision_macro_cnn:.4f}")
print(f"   Recall (Macro)      : {recall_macro_cnn:.4f}")
print(f"   F1-score (Macro)    : {f1_macro_cnn:.4f}")
print("="*70)


Modèle CNN-BiLSTM+Attention sauvegardé dans: models/modele_cnn_bilstm_attention_final.keras
Meilleur modèle sauvegardé dans: models/modele_cnn_bilstm_attention_best.keras

ÉVALUATION SUR LE JEU DE TEST
170/170 ━━━━━━━━━━━━━━━━━━━━ 17s 61ms/step

RÉSULTATS DU MODÈLE 3 (CNN-BiLSTM + Attention)
Hamming Loss (H-Loss) : 0.0392 (Proche de 0 : Mieux)
AUC-ROC (Macro)       : 0.8002 (Proche de 1 : Mieux)
-----------------------------------
   Micro-Averaged (Global/Fréquent)
   Precision (Micro)   : 0.6331
   Recall (Micro)      : 0.1413
   F1-score (Micro)    : 0.2310
-----------------------------------
   Macro-Averaged (Classes Rares)
   Precision (Macro)   : 0.1852
   Recall (Macro)      : 0.0952
   F1-score (Macro)    : 0.1090


In [ ]:
# ==========================================================================
# TEST DE PRÉDICTION SUR DES PHRASES PERSONNALISÉES
# ==========================================================================

def predict_emotions(text, model, tokenizer, threshold=0.5):
    """
    Prédit les émotions pour une phrase donnée.
    
    Args:
        text: La phrase à analyser (string)
        model: Le modèle entraîné (CNN-BiLSTM-Attention)
        tokenizer: Le tokenizer Keras utilisé pour l'entraînement
        threshold: Seuil de décision pour la classification binaire (default: 0.5)
        
    Returns:
        dict: Dictionnaire avec les émotions détectées et leurs probabilités
    """
    # 1. Prétraitement de la phrase
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=MAX_LEN, padding='post', truncating='post')
    
    # 2. Prédiction
    probabilities = model.predict(padded_sequence, verbose=0)[0]
    
    # 3. Extraction des émotions détectées
    detected_emotions = {}
    all_emotions = {}
    
    for i, emotion in enumerate(EMOTION_LABELS):
        prob = probabilities[i]
        all_emotions[emotion] = prob
        if prob >= threshold:
            detected_emotions[emotion] = prob
    
    return detected_emotions, all_emotions


def display_prediction(text, model, tokenizer, threshold=0.5, top_k=5):
    """
    Affiche les prédictions d'émotions pour une phrase de manière formatée.
    
    Args:
        text: La phrase à analyser
        model: Le modèle entraîné
        tokenizer: Le tokenizer Keras
        threshold: Seuil de décision (default: 0.5)
        top_k: Nombre d'émotions les plus probables à afficher (default: 5)
    """
    detected, all_probs = predict_emotions(text, model, tokenizer, threshold)
    
    # Trier les émotions par probabilité décroissante
    sorted_emotions = sorted(all_probs.items(), key=lambda x: x[1], reverse=True)
    
    print("="*70)
    print("PRÉDICTION D'ÉMOTIONS")
    print("="*70)
    print(f"Phrase: \"{text}\"")
    print("-"*70)
    
    if detected:
        print(f"\nÉmotions détectées (seuil >= {threshold}):")
        for emotion, prob in sorted(detected.items(), key=lambda x: x[1], reverse=True):
            print(f"  - {emotion:20s}: {prob:.4f} ({prob*100:.2f}%)")
    else:
        print(f"\nAucune émotion détectée avec un seuil >= {threshold}")
    
    print(f"\nTop {top_k} émotions les plus probables:")
    for i, (emotion, prob) in enumerate(sorted_emotions[:top_k], 1):
        bar = "█" * int(prob * 50)
        print(f"  {i}. {emotion:20s}: {prob:.4f} {bar}")
    
    print("="*70 + "\n")


# ==========================================================================
# EXEMPLES DE TEST
# ==========================================================================

print("\n" + "="*70)
print("TEST DU MODÈLE CNN-BiLSTM-ATTENTION SUR DES PHRASES PERSONNALISÉES")
print("="*70 + "\n")

# Liste de phrases de test
test_phrases = [
    "I am so happy and excited about this!",
    "This is absolutely terrible and makes me angry.",
    "I'm confused and don't know what to do.",
    "Thank you so much! I really appreciate your help.",
    "I miss my family and feel sad.",
    "This is hilarious! LOL",
    "I'm scared and nervous about the exam tomorrow.",
    "What an amazing surprise! I love it!",
    "I feel disappointed and let down.",
    "This is just okay, nothing special."
]

# Tester chaque phrase
for phrase in test_phrases:
    display_prediction(phrase, model_cnn_bilstm_att, tokenizer, threshold=0.3, top_k=5)


# ==========================================================================
# INTERFACE INTERACTIVE POUR TESTER VOS PROPRES PHRASES
# ==========================================================================

print("\n" + "="*70)
print("TEST INTERACTIF - Entrez vos propres phrases")
print("="*70)
print("Instructions: Entrez une phrase en anglais pour prédire ses émotions.")
print("             Tapez 'quit' ou 'exit' pour arrêter.\n")

while True:
    user_input = input("Entrez une phrase: ").strip()
    
    if user_input.lower() in ['quit', 'exit', 'q', '']:
        print("\nFin du test interactif.")
        break
    
    display_prediction(user_input, model_cnn_bilstm_att, tokenizer, threshold=0.3, top_k=5)


# ==========================================================================
# COMPARAISON DES PRÉDICTIONS ENTRE LES MODÈLES
# ==========================================================================

def compare_models(text, threshold=0.3):
    """
    Compare les prédictions de tous les modèles entraînés.
    
    Args:
        text: La phrase à analyser
        threshold: Seuil de décision
    """
    print("="*70)
    print("COMPARAISON DES MODÈLES")
    print("="*70)
    print(f"Phrase: \"{text}\"")
    print("-"*70)
    
    models = [
        ("LSTM Simple", model_lstm_loaded if 'model_lstm_loaded' in globals() else None),
        ("BiLSTM + Attention", model_bilstm_att if 'model_bilstm_att' in globals() else None),
        ("CNN-BiLSTM + Attention", model_cnn_bilstm_att if 'model_cnn_bilstm_att' in globals() else None)
    ]
    
    for model_name, model in models:
        if model is None:
            print(f"\n{model_name}: Modèle non disponible")
            continue
            
        detected, all_probs = predict_emotions(text, model, tokenizer, threshold)
        
        print(f"\n{model_name}:")
        if detected:
            for emotion, prob in sorted(detected.items(), key=lambda x: x[1], reverse=True)[:3]:
                print(f"  - {emotion:20s}: {prob:.4f}")
        else:
            print(f"  Aucune émotion détectée (seuil >= {threshold})")
    
    print("="*70 + "\n")


# Test de comparaison
print("\n" + "="*70)
print("COMPARAISON DES 3 MODÈLES SUR DES PHRASES EXEMPLES")
print("="*70 + "\n")

comparison_phrases = [
    "I'm so grateful and happy!",
    "This makes me really angry and frustrated.",
    "I feel sad and disappointed about this situation."
]

for phrase in comparison_phrases:
    compare_models(phrase, threshold=0.3)

# ⭐⭐  BERT-base pour la classification multi-label des émotions
- Utilise BertTokenizer (WordPiece, pas word-level)
Utilise BertTokenizer (WordPiece, pas word-level)
Ajoute automatiquement les tokens spéciaux [CLS] et [SEP]
Gère l'attention mask pour ignorer le padding
Max length: 128 tokens (suffisant pour les commentaires Reddit)
Comprend le contexte sémantique profond
Pré-entraîné sur des milliards de mots
Meilleure gestion des mots rares (subword tokenization)
Attention multi-têtes pour capturer différentes relations
Performance state-of-the-art sur NLP

In [ ]:
# ========================================================================== #
# MODÈLE 4: BERT-BASE TRANSFORMER (Fine-tuning) - FIX SUBCLASSING API
# ========================================================================== #
import numpy as np
import pandas as pd
import transformers
from transformers import BertTokenizer, TFBertModel, BertConfig
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


print("="*70)
print("MODÈLE 4: BERT-BASE POUR CLASSIFICATION MULTI-LABEL")
print("="*70)
print(f"Version Transformers: {transformers.__version__}")
print(f"Version TensorFlow: {tf.__version__}")

# --- 1. CHARGEMENT DU TOKENIZER ET MODÈLE BERT ---
print("\nChargement du tokenizer BERT...")
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
print("✓ Tokenizer chargé")


# --- 2. CONSTRUCTION DU MODÈLE - SUBCLASSING API (PAS FUNCTIONAL) ---
class BERTMultiLabelClassifier(Model):
    """
    Modèle BERT pour classification multi-label utilisant Subclassing API.
    
    Contrairement à la Functional API, le Subclassing API permet de passer
    directement les KerasTensors à BERT sans problème de conversion.
    """
    
    def __init__(self, num_labels=28, bert_model_name='bert-base-uncased', **kwargs):
        """
        Args:
            num_labels: Nombre de classes (28 émotions)
            bert_model_name: Nom du modèle BERT pré-entraîné
        """
        super().__init__(**kwargs)
        
        # Charger le modèle BERT pré-entraîné
        print(f"\nChargement du modèle BERT {bert_model_name}...")
        self.bert = TFBertModel.from_pretrained(
            bert_model_name,
            use_safetensors=False  # Fix pour safetensors
        )
        print("✓ BERT chargé")
        
        # Couches de classification
        self.dropout_bert = Dropout(0.3)
        self.dense_1 = Dense(256, activation='relu', name='dense_1')
        self.dropout_1 = Dropout(0.3)
        self.dense_2 = Dense(128, activation='relu', name='dense_2')
        self.dropout_2 = Dropout(0.3)
        self.output_layer = Dense(num_labels, activation='sigmoid', name='output')
        
        self.num_labels = num_labels
    
    def call(self, inputs, training=False):
        """
        Forward pass du modèle.
        
        Args:
            inputs: Dict avec 'input_ids' et 'attention_mask'
            training: Booléen pour dropout
        
        Returns:
            Tensor de probabilités (batch_size, num_labels)
        """
        # Extraire les inputs
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        
        # ← CLEF: Passer les tensors TensorFlow directement à BERT
        # BERT accepte les TensorFlow tensors, pas les KerasTensors
        bert_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            training=training
        )
        
        # Utiliser le token [CLS] (pooled output)
        cls_token = bert_output[1]  # Shape: (batch_size, 768)
        
        # Classification layers
        x = self.dropout_bert(cls_token, training=training)
        x = self.dense_1(x)
        x = self.dropout_1(x, training=training)
        x = self.dense_2(x)
        x = self.dropout_2(x, training=training)
        output = self.output_layer(x)
        
        return output
    
    def get_config(self):
        """Pour pouvoir sauvegarder et charger le modèle"""
        return {
            'num_labels': self.num_labels,
            'bert_model_name': 'bert-base-uncased'
        }


# --- 3. CRÉER UNE INSTANCE DU MODÈLE ---
print("\n" + "="*70)
print("CONSTRUCTION DU MODÈLE BERT")
print("="*70)

NUM_LABELS = 28
MAX_LENGTH_BERT = 128

model_bert = BERTMultiLabelClassifier(
    num_labels=NUM_LABELS,
    bert_model_name='bert-base-uncased',
    name='BERT_MultiLabel_Classifier'
)

print("✓ Modèle créé avec succès!")


# --- 4. COMPILATION DU MODÈLE ---
print("\n" + "="*70)
print("COMPILATION DU MODÈLE")
print("="*70)

def custom_loss_function(y_true, y_pred):
    """
    Weighted Binary Cross-Entropy pour multi-label.
    """
    y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
    bce = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)
    return tf.reduce_mean(bce)

model_bert.compile(
    optimizer=Adam(learning_rate=2e-5),
    loss=custom_loss_function,
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc', multi_label=True),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

print("✓ Modèle compilé avec succès")
print("  - Optimizer: Adam (lr=2e-5)")
print("  - Loss: Weighted Binary Cross-Entropy")
print("  - Metrics: AUC, Precision, Recall")


# --- 5. RÉSUMÉ DE L'ARCHITECTURE ---
print("\n" + "="*70)
print("ARCHITECTURE DU MODÈLE")
print("="*70)

# Build the model by calling it once
dummy_input = {
    'input_ids': tf.zeros((1, MAX_LENGTH_BERT), dtype=tf.int32),
    'attention_mask': tf.ones((1, MAX_LENGTH_BERT), dtype=tf.int32)
}
_ = model_bert(dummy_input, training=False)

print(model_bert.summary())


# --- 6. FONCTION DE TOKENIZATION ---
def encode_texts_for_bert(texts, tokenizer, max_length=128):
    """
    Tokenise les textes au format BERT.
    
    Args:
        texts: Liste de textes
        tokenizer: BertTokenizer
        max_length: Longueur maximale
    
    Returns:
        tuple: (input_ids, attention_mask) comme numpy arrays
    """
    input_ids = []
    attention_masks = []
    
    for text in texts:
        if not isinstance(text, str):
            text = str(text)
        
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors=None  # Pas de tensors
        )
        
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    return np.array(input_ids), np.array(attention_masks)


# --- 7. CRÉER LES DATASETS ---
print("\n" + "="*70)
print("PRÉPARATION DES DONNÉES")
print("="*70)

# Créer des données fictices pour démonstration
np.random.seed(42)
n_samples_train = 100
n_samples_dev = 20
n_samples_test = 20

# Textes
texts_train = [f"This is sample text number {i} for training" for i in range(n_samples_train)]
texts_dev = [f"This is sample text number {i} for development" for i in range(n_samples_dev)]
texts_test = [f"This is sample text number {i} for testing" for i in range(n_samples_test)]

# Labels multi-label
y_train = np.random.randint(0, 2, size=(n_samples_train, NUM_LABELS)).astype(np.float32)
y_dev = np.random.randint(0, 2, size=(n_samples_dev, NUM_LABELS)).astype(np.float32)
y_test = np.random.randint(0, 2, size=(n_samples_test, NUM_LABELS)).astype(np.float32)

print("\nTokenisation des données...")
train_input_ids, train_attention_mask = encode_texts_for_bert(texts_train, bert_tokenizer, MAX_LENGTH_BERT)
dev_input_ids, dev_attention_mask = encode_texts_for_bert(texts_dev, bert_tokenizer, MAX_LENGTH_BERT)
test_input_ids, test_attention_mask = encode_texts_for_bert(texts_test, bert_tokenizer, MAX_LENGTH_BERT)

print(f"✓ input_ids shape: {train_input_ids.shape}")
print(f"✓ attention_mask shape: {train_attention_mask.shape}")
print(f"✓ labels shape: {y_train.shape}")

# Créer les datasets TensorFlow
BATCH_SIZE = 16

def create_dataset(input_ids, attention_mask, labels, batch_size=32, shuffle=True):
    """Crée un tf.data.Dataset."""
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': input_ids,
            'attention_mask': attention_mask
        },
        labels
    ))
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(labels), reshuffle_each_iteration=True)
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset

train_dataset = create_dataset(train_input_ids, train_attention_mask, y_train, BATCH_SIZE)
dev_dataset = create_dataset(dev_input_ids, dev_attention_mask, y_dev, BATCH_SIZE, shuffle=False)
test_dataset = create_dataset(test_input_ids, test_attention_mask, y_test, BATCH_SIZE, shuffle=False)

print(f"\n✓ Train dataset: {len(train_dataset)} batches")
print(f"✓ Dev dataset: {len(dev_dataset)} batches")
print(f"✓ Test dataset: {len(test_dataset)} batches")


# --- 8. TEST FORWARD PASS AVANT ENTRAÎNEMENT ---
print("\n" + "="*70)
print("TEST FORWARD PASS")
print("="*70)

print("\nFaisant une prédiction de test...")
sample_batch = next(iter(train_dataset))
sample_input, sample_label = sample_batch

try:
    prediction = model_bert(sample_input, training=False)
    print(f"✓ Forward pass réussi!")
    print(f"  Input shape: {sample_input['input_ids'].shape}")
    print(f"  Output shape: {prediction.shape}")
    print(f"  Output range: [{prediction.numpy().min():.4f}, {prediction.numpy().max():.4f}]")
except Exception as e:
    print(f"✗ Erreur lors du forward pass: {str(e)}")
    raise


# --- 9. CALLBACKS POUR L'ENTRAÎNEMENT ---
import os
os.makedirs('models', exist_ok=True)

callbacks_bert = [
    EarlyStopping(
        monitor='val_auc',
        patience=3,
        mode='max',
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        filepath='models/modele_bert_best.keras',
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-7,
        verbose=1
    )
]


# --- 10. ENTRAÎNEMENT ---
print("\n" + "="*70)
print("ENTRAÎNEMENT DU MODÈLE")
print("="*70)

history_bert = model_bert.fit(
    train_dataset,
    validation_data=dev_dataset,
    epochs=5,  # Augmenter pour de meilleurs résultats
    callbacks=callbacks_bert,
    verbose=1
)

print("\n✓ Entraînement terminé!")


# --- 11. ÉVALUATION ---
print("\n" + "="*70)
print("ÉVALUATION SUR L'ENSEMBLE DE TEST")
print("="*70)

test_results = model_bert.evaluate(test_dataset, verbose=1)
print(f"\nRésultats de test:")
print(f"  Loss: {test_results[0]:.4f}")
print(f"  Accuracy: {test_results[1]:.4f}")
print(f"  AUC: {test_results[2]:.4f}")
print(f"  Precision: {test_results[3]:.4f}")
print(f"  Recall: {test_results[4]:.4f}")


# --- 12. SAUVEGARDE ET CHARGEMENT ---
print("\n" + "="*70)
print("SAUVEGARDE DU MODÈLE")
print("="*70)

# Sauvegarder le modèle
model_bert.save('models/bert_model_final.keras')
print("✓ Modèle sauvegardé: models/bert_model_final.keras")

# Sauvegarder le tokenizer
bert_tokenizer.save_pretrained('models/bert_tokenizer')
print("✓ Tokenizer sauvegardé: models/bert_tokenizer")


# --- 13. PRÉDICTIONS SUR NOUVEAUX TEXTES ---
print("\n" + "="*70)
print("PRÉDICTIONS SUR NOUVEAUX TEXTES")
print("="*70)

def predict_emotions(texts, model, tokenizer, max_length=128, threshold=0.5):
    """
    Prédit les émotions pour une liste de textes.
    
    Args:
        texts: Liste de textes
        model: Modèle entraîné
        tokenizer: Tokenizer BERT
        max_length: Longueur maximale
        threshold: Seuil de prédiction
    
    Returns:
        tuple: (probabilités, prédictions binaires)
    """
    input_ids, attention_mask = encode_texts_for_bert(texts, tokenizer, max_length)
    
    # Créer un dataset avec batch_size = nombre de textes
    dataset = tf.data.Dataset.from_tensor_slices({
        'input_ids': input_ids,
        'attention_mask': attention_mask
    }).batch(len(texts))
    
    # Faire les prédictions
    batch = next(iter(dataset))
    predictions = model(batch, training=False)
    
    # Appliquer le threshold
    binary_predictions = (predictions.numpy() > threshold).astype(int)
    
    return predictions.numpy(), binary_predictions

# Exemple de prédiction
test_texts = [
    "I love this movie! It's amazing and beautiful.",
    "This is terrible and makes me very sad.",
    "I am not sure about this situation."
]

print("\nTextes de test:")
for i, text in enumerate(test_texts, 1):
    print(f"  {i}. {text}")

probs, binary_preds = predict_emotions(test_texts, model_bert, bert_tokenizer)

print(f"\nProbabilités prédites shape: {probs.shape}")
print(f"Prédictions binaires shape: {binary_preds.shape}")
print("\nExemple de prédictions (5 premières émotions):")
for i, text in enumerate(test_texts):
    print(f"\nTexte {i+1}: {text}")
    print(f"  Probabilités: {probs[i][:5]}")
    print(f"  Prédictions: {binary_preds[i][:5]}")


# --- 14. CHARGER LE MODÈLE SAUVEGARDÉ ---
print("\n" + "="*70)
print("CHARGEMENT DU MODÈLE SAUVEGARDÉ")
print("="*70)

print("\nChargement du modèle depuis le disque...")
try:
    # Charger le modèle
    loaded_model = tf.keras.models.load_model('models/bert_model_final.keras')
    print("✓ Modèle chargé avec succès!")
    
    # Charger le tokenizer
    loaded_tokenizer = BertTokenizer.from_pretrained('models/bert_tokenizer')
    print("✓ Tokenizer chargé avec succès!")
    
    # Tester une prédiction
    test_text = ["Hello, this is a test!"]
    probs_loaded, _ = predict_emotions(test_text, loaded_model, loaded_tokenizer)
    print(f"✓ Prédiction avec modèle chargé réussie!")
    print(f"  Probabilités: {probs_loaded[0][:5]}")
    
except Exception as e:
    print(f"✗ Erreur lors du chargement: {str(e)}")


print("\n" + "="*70)
print("FIN DU SCRIPT")
print("="*70)

MODÈLE 4: BERT-BASE POUR CLASSIFICATION MULTI-LABEL
Version Transformers: 4.57.3
Version TensorFlow: 2.20.0

Chargement du tokenizer BERT...
✓ Tokenizer chargé

CONSTRUCTION DU MODÈLE BERT

Chargement du modèle BERT bert-base-uncased...


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


✓ BERT chargé
✓ Modèle créé avec succès!

COMPILATION DU MODÈLE
✓ Modèle compilé avec succès
  - Optimizer: Adam (lr=2e-5)
  - Loss: Weighted Binary Cross-Entropy
  - Metrics: AUC, Precision, Recall

ARCHITECTURE DU MODÈLE


Model: "BERT_MultiLabel_Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (1, 256)               │       196,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (1, 128)               │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (1, 28)                │         3,612 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 233,372 (911.61 KB)

 Trainable params: 233,372 (911.61 KB)

 Non-trainable params: 0 (0.00 B)

None

PRÉPARATION DES DONNÉES

Tokenisation des données...
✓ input_ids shape: (100, 128)
✓ attention_mask shape: (100, 128)
✓ labels shape: (100, 28)

✓ Train dataset: 7 batches
✓ Dev dataset: 2 batches
✓ Test dataset: 2 batches

TEST FORWARD PASS

Faisant une prédiction de test...
✓ Forward pass réussi!
  Input shape: (16, 128)
  Output shape: (16, 28)
  Output range: [0.2081, 0.7551]

ENTRAÎNEMENT DU MODÈLE
Epoch 1/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.0644 - auc: 0.5111 - loss: 0.7909 - precision: 0.5070 - recall: 0.4356

KeyboardInterrupt: 